# Downloading documents from Wikipedia

In [4]:
import requests
import bs4
import re
import tqdm
from collections import deque
from time import sleep
import random
import pandas as pd

In [63]:
def generator(texts):
  while len(texts) < 1000:
    yield


In [64]:
# dodać graf dla ciekawości i zwiększyć ilość stron
def bfs(link):
    response = requests.get(link)
    parsed = bs4.BeautifulSoup(response.text)
    links = parsed.find_all(
        'a', attrs={'href': re.compile(r'^/wiki')})  # english only
    texts = {link: parsed}
    q = deque()
    for a in links:
        q.append("https://en.wikipedia.org" + a['href'])
    for _ in (pbar := tqdm.tqdm(generator(texts))):
        pbar.set_description(f'{len(texts)} sites already')
        v = q.popleft()
        response = requests.get(v)
        if response.status_code != 200:
            continue
        parsed = bs4.BeautifulSoup(response.text)
        links = parsed.find_all(
            'a', attrs={'href': re.compile(r'^/wiki')})
        texts[v] = parsed
        for a in links:
            q.append("https://en.wikipedia.org" + a['href'])
        sleep(random.random()*3)
    return texts


In [65]:
texts = bfs('https://en.wikipedia.org/wiki/Jazz')


999 sites already: : 1445it [41:19,  1.72s/it]


In [66]:
df = pd.DataFrame([texts]).transpose()
df.to_csv('raw_texts.csv')
df


,0
https://en.wikipedia.org/wiki/Jazz,"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
https://en.wikipedia.org/wiki/Wikipedia:Protection_policy#semi,"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
https://en.wikipedia.org/wiki/Jazz_(disambiguation),"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
https://en.wikipedia.org/wiki/Blues,"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
https://en.wikipedia.org/wiki/Ragtime,"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
...,...
https://en.wikipedia.org/wiki/Sophisti-pop,"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
https://en.wikipedia.org/wiki/Music_of_West_Africa,"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
https://en.wikipedia.org/wiki/Bird_(1988_film),"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."
https://en.wikipedia.org/wiki/Round_Midnight_(film),"[html, [\n, [\n, <meta charset=""utf-8""/>, \n, ..."


In [67]:
print(len(set(texts.values())))

994
